In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_path = '/content/drive/MyDrive/vicuna'

In [ ]:
!pip install transformers transformers[sentencepiece]
!pip install ipywidgets==7.7.1

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install accelerate

In [ ]:
!pip install fschat

In [ ]:
import json

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

#

In [ ]:
#from fastchat.model import load_model, get_conversation_template, add_model_args
# model_path = 'lmsys/vicuna-13b-v1.3'
# model, tokenizer = load_model(
#     model_path=model_path,
#     device='cuda',
#     num_gpus=1,
#     #load_8bit=True
#     #cache_dir=f"{base_path}/{model_path}"
# )

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model_path = "meta-llama/Llama-2-13b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_path)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_path,
    temperature=0.1,
    torch_dtype=torch.float16,
    device_map="auto",
)


In [ ]:
def get_llama_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """

    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512
    )
    return sequences[0]['generated_text']

In [ ]:
prompt = """Create a comma-separated list of all medical terms in the original language without translations in the medical article below:
"{0}"
"""

prompt_translate = """Translate the following medical terms in English and create a comma-separated list of their standard UMLS names:
{0}
"""

#F1 - ?

In [ ]:
max_new_tokens= 512
repetition_penalty = 1.0
temperature = 0.1

In [ ]:
text = "Pseudomonas aeruginosa (Pa) infection in cystic fibrosis (CF) patients is associated with worse long-term pulmonary disease and shorter survival, and chronic Pa infection (CPA) is associated with reduced lung function, faster rate of lung decline, increased rates of exacerbations and shorter survival."
#text = "DCTN4 as a modifier of chronic Pseudomonas aeruginosa infection in cystic fibrosis. Pseudomonas aeruginosa (Pa) infection in cystic fibrosis (CF) patients is associated with worse long-term pulmonary disease and shorter survival, and chronic Pa infection (CPA) is associated with reduced lung function, faster rate of lung decline, increased rates of exacerbations and shorter survival. By using exome sequencing and extreme phenotype design, it was recently shown that isoforms of dynactin 4 (DCTN4) may influence Pa infection in CF, leading to worse respiratory disease. The purpose of this study was to investigate the role of DCTN4 missense variants on Pa infection incidence, age at first Pa infection and chronic Pa infection incidence in a cohort of adult CF patients from a single centre. Polymerase chain reaction and direct sequencing were used to screen DNA samples for DCTN4 variants. A total of 121 adult CF patients from the Cochin Hospital CF centre have been included, all of them carrying two CFTR defects: 103 developed at least 1 pulmonary infection with Pa, and 68 patients of them had CPA. DCTN4 variants were identified in 24% (29/121) CF patients with Pa infection and in only 17% (3/18) CF patients with no Pa infection. Of the patients with CPA, 29% (20/68) had DCTN4 missense variants vs 23% (8/35) in patients without CPA. Interestingly, p.Tyr263Cys tend to be more frequently observed in CF patients with CPA than in patients without CPA (4/68 vs 0/35), and DCTN4 missense variants tend to be more frequent in male CF patients with CPA bearing two class II mutations than in male CF patients without CPA bearing two class II mutations (P = 0.06). Our observations reinforce that DCTN4 missense variants, especially p.Tyr263Cys, may be involved in the pathogenesis of CPA in male CF."

In [ ]:
msg = prompt.format(text)
msg

In [ ]:
def get_vicuna_response(text, prompt_text):
    msg = prompt_text.format(text)
    conv = get_conversation_template(model_path)
    conv.append_message(conv.roles[0], msg)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=True,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        max_new_tokens=max_new_tokens,
    )

    if model.config.is_encoder_decoder:
        output_ids = output_ids[0]
    else:
        output_ids = output_ids[0][len(input_ids[0]) :]
    outputs = tokenizer.decode(
        output_ids, skip_special_tokens=True, spaces_between_special_tokens=False
    )

    print(f"{conv.roles[0]}: {msg}")
    print(f"{conv.roles[1]}: {outputs}")

    #return convert_output(outputs)
    return outputs

In [ ]:
def get_vicuna_response_multi(msg, previous_prompts=[], previous_responses=[], max_new_tokens=512):
    conv = get_conversation_template(model_path)

    for prompt, response in zip(previous_prompts, previous_responses):
        conv.append_message(conv.roles[0], prompt)
        conv.append_message(conv.roles[1], response)

    conv.append_message(conv.roles[0], msg)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=True,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        max_new_tokens=max_new_tokens,
    )

    if model.config.is_encoder_decoder:
        output_ids = output_ids[0]
    else:
        output_ids = output_ids[0][len(input_ids[0]) :]
    outputs = tokenizer.decode(
        output_ids, skip_special_tokens=True, spaces_between_special_tokens=False
    )

    print(f"{conv.roles[0]}: {msg}")
    print(f"{conv.roles[1]}: {outputs}")

    #return convert_output(outputs)
    return outputs

In [ ]:
#outputs = get_vicuna_response(text, prompt)
text = "дългогодишна хипертония , има често главоболие , сърцебиене и лесно заморяване продължават оплакванията от стягане в сърдечната област , сърцебиене , гадене . доказан карцином на простатата . насочва се за хоспитализация за химиотерапия ."
msg = prompt.format(text)
outputs = get_llama_response(msg)
outputs

In [ ]:
result = outputs.replace(msg, '')

if ':\n' in result:
  result = result[result.index(':\n')+2:]

print(result)

In [ ]:
translate_msg = prompt_translate.format(result)
outputs = get_llama_response(translate_msg)
outputs.replace(translate_msg, '')

In [ ]:
text = """Nonylphenol diethoxylate inhibits apoptosis induced in PC12 cells
Nonylphenol and short-chain nonylphenol ethoxylates such as NP2 EO are present in aquatic environment as wastewater contaminants, and their toxic effects on aquatic species have been reported. Apoptosis has been shown to be induced by serum deprivation or copper treatment. To understand the toxicity of nonylphenol diethoxylate, we investigated the effects of NP2 EO on apoptosis induced by serum deprivation and copper by using PC12 cell system. Nonylphenol diethoxylate itself showed no toxicity and recovered cell viability from apoptosis. In addition, nonylphenol diethoxylate decreased DNA fragmentation caused by apoptosis in PC12 cells."""
#outputs = get_vicuna_response(text, prompt)

In [ ]:
text = """DCTN4 as a modifier of chronic Pseudomonas aeruginosa infection in cystic fibrosis
Pseudomonas aeruginosa (Pa) infection in cystic fibrosis (CF) patients is associated with worse long-term pulmonary disease and shorter survival, and chronic Pa infection (CPA) is associated with reduced lung function, faster rate of lung decline, increased rates of exacerbations and shorter survival. By using exome sequencing and extreme phenotype design, it was recently shown that isoforms of dynactin 4 (DCTN4) may influence Pa infection in CF, leading to worse respiratory disease. The purpose of this study was to investigate the role of DCTN4 missense variants on Pa infection incidence, age at first Pa infection and chronic Pa infection incidence in a cohort of adult CF patients from a single centre."""

In [ ]:
len(text)

In [ ]:
import pandas as pd

df_sentences = pd.read_csv(f'{base_path}/st21pv_test_sentences_combined.tsv', sep='\t', header=0)
#df_sentences = pd.read_csv(f'{base_path}/bg_texts_30.txt', sep='\t', header=None)
#df_sentences.columns = ['sentence']
df_sentences.head()

In [ ]:
df_sentences.iloc[13]['sentence']

In [ ]:
df_sentences.shape

In [ ]:
#import pandas as pd

#df_sentences = pd.read_csv('MedMentions-master/MedMentions-master/bg_texts_30.txt', sep='\t', header=None)
#df_sentences.columns = ['sentence']
#df_sentences.head()

In [ ]:
def store_checkpoint(result_outputs, path):
    df_checkpoint = pd.DataFrame({'outputs': result_outputs })
    df_checkpoint.to_csv(f'{base_path}/{path}.tsv', sep='\t', index=False)

In [ ]:
checkpoint_file = "df_checkpoint_sp21_test_sentences_llama_prompt_0929_v1_13b" # "df_checkpoint_bg_test_sentences_prompt_0928_v1_13b"

In [ ]:
import datetime

texts = []
result_outputs = []
batch_size = 10

print(datetime.datetime.now())
for index, row in df_sentences.iterrows():
    #print(row)
    text = row['sentence']
    print(text)
    texts.append(text)
    msg = prompt.format(text)
    prompts = [msg]
    responses = []

    outputs = get_llama_response(msg)
    result = outputs.replace(msg, '')

    if ':\n' in result:
      result = result[result.index(':\n')+2:]

    responses.append(outputs) # store full output

    translate_msg = prompt_translate.format(result)
    translate_outputs = get_llama_response(translate_msg)

    prompts.append(translate_msg)
    responses.append(translate_outputs)

    result_outputs.append(responses)

    if index % batch_size == 0:
        store_checkpoint(result_outputs, path=checkpoint_file)
        print(f"Store checkpoint {index} at {datetime.datetime.now()}")
print(datetime.datetime.now())

In [ ]:
import datetime

texts = []
result_outputs = []
batch_size = 10

print(datetime.datetime.now())
for index, row in df_sentences.iterrows():
    #print(row)
    text = row['sentence']
    print(text)
    texts.append(text)
    msg = prompt.format(text)
    outputs = get_llama_response(msg)
    print(outputs)
    result_outputs.append(outputs)

    if index % batch_size == 0:
        store_checkpoint(result_outputs, path=checkpoint_file)
        print(f"Store checkpoint {index} at {datetime.datetime.now()}")
print(datetime.datetime.now())

In [ ]:
# import datetime

# texts = []
# result_outputs = []
# batch_size = 10

# print(datetime.datetime.now())
# for index, row in df_sentences.iterrows():
#     #print(row)
#     text = row['sentence']
#     print(text)
#     texts.append(text)
#     msg = prompt.format(text)
#     prompts = [msg]
#     responses = []
#     outputs = get_vicuna_response_multi(msg)
#     responses.append(outputs)

#     translate_msg = prompt_translate.format(outputs)
#     translate_outputs = get_vicuna_response_multi(translate_msg, prompts, responses, max_new_tokens=512)
#     prompts.append(translate_msg)
#     responses.append(translate_outputs)

#     result_outputs.append(responses)

#     if index % batch_size == 0:
#         store_checkpoint(result_outputs, path=checkpoint_file)
#         print(f"Store checkpoint {index} at {datetime.datetime.now()}")
# print(datetime.datetime.now())

In [ ]:
# import datetime

# texts = []
# result_outputs = []
# batch_size = 10

# print(datetime.datetime.now())
# for index, row in df_sentences.iterrows():
#     #print(row)
#     text = row['sentence']
#     print(text)
#     texts.append(text)
#     outputs = get_vicuna_response(text, prompt)
#     result_outputs.append(outputs)

#     if index % batch_size == 0:
#         store_checkpoint(result_outputs, path=checkpoint_file)
#         print(f"Store checkpoint {index} at {datetime.datetime.now()}")
# print(datetime.datetime.now())

In [ ]:
len(result_outputs)

In [ ]:
store_checkpoint(result_outputs, path=checkpoint_file)